## Практическая работа

### Задание 1

Рассчитать бизнес-метрики: GMV и Revenue.

Используйте датасет transactions.tsv, содержащий информацию, какие книги покупали пользователи.

- Необходимо посчитать значения метрик для каждого года.
- В столбце Price указана полная цена книги. Для нахождения Revenue считайте, что на протяжении всего периода на все имеющиеся книги действовала скидка в 10%.

In [1]:
## + Добавить скриншот с формулой из готовой презентации
## + Пересчитать Precision для К = 5 и обновлённого предикта 

In [3]:
import pandas as pd
import numpy as np

In [5]:
data = pd.read_csv('transactions.tsv.gz', sep = '\t')
data

,product_id,ISBN,Book-Rating,User-ID,Book-Title,picture_url,date,price
0,10397,3404139178,9,86583,Das Lacheln der Fortuna: Historischer Roman,http://images.amazon.com/images/P/3404139178.0...,20/06/2022 06:50 AM,1890
1,10397,3404139178,10,132500,Das Lacheln der Fortuna: Historischer Roman,http://images.amazon.com/images/P/3404139178.0...,15/01/2024 10:52 AM,1890
2,10397,3404139178,10,66483,Das Lacheln der Fortuna: Historischer Roman,http://images.amazon.com/images/P/3404139178.0...,05/01/2023 12:38 PM,1890
3,10397,3404139178,9,276866,Das Lacheln der Fortuna: Historischer Roman,http://images.amazon.com/images/P/3404139178.0...,03/11/2020 03:28 PM,1890
4,10397,3404139178,6,106534,Das Lacheln der Fortuna: Historischer Roman,http://images.amazon.com/images/P/3404139178.0...,17/02/2021 03:34 PM,1890
...,...,...,...,...,...,...,...,...
110514,9478,0849912970,10,27551,Traveling Light: Releasing the Burdens You Wer...,http://images.amazon.com/images/P/0849912970.0...,18/03/2020 08:59 PM,2080
110515,10209,1860465811,9,206652,Wind-Up Bird Chronicle,http://images.amazon.com/images/P/1860465811.0...,28/06/2017 03:17 PM,1020
110516,10138,1588720284,8,223525,Why Your Life Sucks: And What You Can Do Abou...,http://images.amazon.com/images/P/1588720284.0...,06/01/2018 02:42 PM,230
110517,276,0060652926,9,94153,Mere Christianity,http://images.amazon.com/images/P/0060652926.0...,30/12/2023 10:18 PM,430


1. Извлечём год из даты

In [14]:
data['year'] = pd.DatetimeIndex(data['date']).year
data.head(3)

,product_id,ISBN,Book-Rating,User-ID,Book-Title,picture_url,date,price,year
0,10397,3404139178,9,86583,Das Lacheln der Fortuna: Historischer Roman,http://images.amazon.com/images/P/3404139178.0...,20/06/2022 06:50 AM,1890,2022
1,10397,3404139178,10,132500,Das Lacheln der Fortuna: Historischer Roman,http://images.amazon.com/images/P/3404139178.0...,15/01/2024 10:52 AM,1890,2024
2,10397,3404139178,10,66483,Das Lacheln der Fortuna: Historischer Roman,http://images.amazon.com/images/P/3404139178.0...,05/01/2023 12:38 PM,1890,2023


2. Сгруппируем по годам, посчитав сумму столбца price

In [21]:
result = data.groupby(['year'])['price'].sum().reset_index()
result

,year,price
0,2017,24096560
1,2018,24117010
2,2019,24472580
3,2020,23736490
4,2021,24137030
5,2022,23941640
6,2023,23437620
7,2024,8138590


3. Чтобы посчитать revenue, нужно найти фактическую цену проданных книг. Для всех продаж действовала скидка в 10%, значит, мы доход составил 0.9 от прибыли

In [23]:
result = result.rename(columns = {'price': 'gmv'})

In [24]:
result['revenue'] = result['gmv'] * 0.9
result

,year,gmv,revenue
0,2017,24096560,21686904.0
1,2018,24117010,21705309.0
2,2019,24472580,22025322.0
3,2020,23736490,21362841.0
4,2021,24137030,21723327.0
5,2022,23941640,21547476.0
6,2023,23437620,21093858.0
7,2024,8138590,7324731.0


------

.

### Задание 2

Посчитать Precision, Precision@K, MAP@K для К = 5

Используйте датасет recoms.tsv, содержащий рекомендации для каждого пользователя, и датасет transactions.tsv, где лежат данные о фактических покупках.

In [4]:
transactions = pd.read_csv('transactions.tsv.gz', sep = '\t')
transactions.head(3)

,product_id,ISBN,Book-Rating,User-ID,Book-Title,picture_url,date,price
0,10397,3404139178,9,86583,Das Lacheln der Fortuna: Historischer Roman,http://images.amazon.com/images/P/3404139178.0...,20/06/2022 06:50 AM,1890
1,10397,3404139178,10,132500,Das Lacheln der Fortuna: Historischer Roman,http://images.amazon.com/images/P/3404139178.0...,15/01/2024 10:52 AM,1890
2,10397,3404139178,10,66483,Das Lacheln der Fortuna: Historischer Roman,http://images.amazon.com/images/P/3404139178.0...,05/01/2023 12:38 PM,1890


In [5]:
recoms = pd.read_csv('recoms.tsv.gz', sep = '\t')
recoms

,User,Book_ids
0,8,"[7080, 7057, 1595, 3981, 7710, 4469, 7908]"
1,99,"[7470, 1444, 4957, 7515, 4849, 9769, 181]"
2,114,"[7791, 500, 2000, 5929, 9114, 3396, 5285]"
3,242,"[952, 4643, 5455, 3800, 7071, 9358, 6041]"
4,243,"[3719, 2771, 8517, 2559, 8892, 7946, 1003]"
...,...,...
10955,278668,"[3824, 5505, 6017, 8314, 7951, 2153, 5803]"
10956,278694,"[3461, 8610, 6399, 8870, 7599, 4412, 1962]"
10957,278843,"[4770, 1783, 2574, 5958, 3925, 8219, 4245]"
10958,278851,"[8680, 5024, 3575, 6505, 1645, 8110, 193]"


1. Найдём книги, которые пользователь действительно купил

In [6]:
purchases = transactions.groupby(['User-ID'])['product_id'].apply(set).reset_index()
purchases['recoms'] = recoms['Book_ids'].apply(lambda x: list(map(int, x[1:-1].split(','))))
purchases

,User-ID,product_id,recoms
0,8,{0},"[7080, 7057, 1595, 3981, 7710, 4469, 7908]"
1,99,"{5415, 1386, 1710, 5105, 1393}","[7470, 1444, 4957, 7515, 4849, 9769, 181]"
2,114,"{5064, 5067, 6956, 5014, 7386, 5628}","[7791, 500, 2000, 5929, 9114, 3396, 5285]"
3,242,"{6529, 10354, 10379, 10526}","[952, 4643, 5455, 3800, 7071, 9358, 6041]"
4,243,"{4544, 4992, 4834, 8900, 294, 3273, 5162, 1743...","[3719, 2771, 8517, 2559, 8892, 7946, 1003]"
...,...,...,...
10955,278668,{10605},"[3824, 5505, 6017, 8314, 7951, 2153, 5803]"
10956,278694,"{8568, 8634, 5685, 6111}","[3461, 8610, 6399, 8870, 7599, 4412, 1962]"
10957,278843,"{4352, 3747, 8804, 7880, 8937, 6507, 111, 240,...","[4770, 1783, 2574, 5958, 3925, 8219, 4245]"
10958,278851,"{9663, 9907, 4214, 4607}","[8680, 5024, 3575, 6505, 1645, 8110, 193]"


2. Найдём множество верно предсказанных элементов

In [75]:
purchases['true_positive'] = purchases.apply(lambda x: set.intersection(x['product_id'], x['recoms']), axis = 1)
purchases

,User-ID,product_id,recoms,true_positive
0,8,{0},"[0, 666]",{0}
1,99,"{5415, 1386, 1710, 5105, 1393}","[5415, 1386, 666]","{1386, 5415}"
2,114,"{5064, 5067, 6956, 5014, 7386, 5628}","[5064, 5067, 666]","{5064, 5067}"
3,242,"{6529, 10354, 10379, 10526}","[6529, 10354, 666]","{6529, 10354}"
4,243,"{4544, 4992, 4834, 8900, 294, 3273, 5162, 1743...","[4544, 4992, 666]","{4544, 4992}"
...,...,...,...,...
10955,278668,{10605},"[10605, 666]",{10605}
10956,278694,"{8568, 8634, 5685, 6111}","[8568, 8634, 666]","{8568, 8634}"
10957,278843,"{4352, 3747, 8804, 7880, 8937, 6507, 111, 240,...","[4352, 3747, 666]","{4352, 3747}"
10958,278851,"{9663, 9907, 4214, 4607}","[9663, 9907, 666]","{9907, 9663}"


3. Посчитаем Рrecision

In [76]:
purchases['precision'] = purchases.apply(lambda x: len(x['true_positive']) * 1.0 /len(x['recoms']), axis = 1)
purchases.head(3)

,User-ID,product_id,recoms,true_positive,precision
0,8,{0},"[0, 666]",{0},0.500000
1,99,"{5415, 1386, 1710, 5105, 1393}","[5415, 1386, 666]","{1386, 5415}",0.666667
2,114,"{5064, 5067, 6956, 5014, 7386, 5628}","[5064, 5067, 666]","{5064, 5067}",0.666667


4. Найдём Precision@5

In [78]:
purchases['true_positive@5'] = purchases.apply(lambda x: set.intersection(x['product_id'], x['recoms'][:5]), axis = 1)
purchases.head(3)

,User-ID,product_id,recoms,true_positive,precision,true_positive@4
0,8,{0},"[0, 666]",{0},0.500000,{0}
1,99,"{5415, 1386, 1710, 5105, 1393}","[5415, 1386, 666]","{1386, 5415}",0.666667,"{1386, 5415}"
2,114,"{5064, 5067, 6956, 5014, 7386, 5628}","[5064, 5067, 666]","{5064, 5067}",0.666667,"{5064, 5067}"


In [79]:
purchases['precision@5'] = purchases.apply(lambda x: len(x['true_positive@5']) * 1.0 /len(x['recoms']), axis = 1  )
purchases.head(3)

,User-ID,product_id,recoms,true_positive,precision,true_positive@4,precision@4
0,8,{0},"[0, 666]",{0},0.500000,{0},0.500000
1,99,"{5415, 1386, 1710, 5105, 1393}","[5415, 1386, 666]","{1386, 5415}",0.666667,"{1386, 5415}",0.666667
2,114,"{5064, 5067, 6956, 5014, 7386, 5628}","[5064, 5067, 666]","{5064, 5067}",0.666667,"{5064, 5067}",0.666667


In [109]:
# Считаем метрики для каждого К

def Precision_top_k(true, predict, K):
    K = min(K, len(true), len(predict))
    res = []
    for i in range(1, K + 1):        
        res.append(len(set.intersection(set(predict[:i]), set(true))) * 1.0 / i)
    return res

In [115]:
purchases['ap@5'] = purchases.apply(lambda x: Precision_top_k(x['product_id'], x['recoms'], 5)  , axis = 1  )
purchases.head(3)

,User-ID,product_id,recoms,true_positive,precision,true_positive@4,precision@4,ap@4
0,8,{0},"[0, 666]",{0},0.500000,{0},0.500000,[1.0]
1,99,"{5415, 1386, 1710, 5105, 1393}","[5415, 1386, 666]","{1386, 5415}",0.666667,"{1386, 5415}",0.666667,"[1.0, 1.0, 0.6666666666666666]"
2,114,"{5064, 5067, 6956, 5014, 7386, 5628}","[5064, 5067, 666]","{5064, 5067}",0.666667,"{5064, 5067}",0.666667,"[1.0, 1.0, 0.6666666666666666]"


In [117]:
purchases['ap@5'] = purchases['ap@5'].apply(lambda x: np.array(x).mean())
purchases

,User-ID,product_id,recoms,true_positive,precision,true_positive@4,precision@4,ap@4
0,8,{0},"[0, 666]",{0},0.500000,{0},0.500000,1.000000
1,99,"{5415, 1386, 1710, 5105, 1393}","[5415, 1386, 666]","{1386, 5415}",0.666667,"{1386, 5415}",0.666667,0.888889
2,114,"{5064, 5067, 6956, 5014, 7386, 5628}","[5064, 5067, 666]","{5064, 5067}",0.666667,"{5064, 5067}",0.666667,0.888889
3,242,"{6529, 10354, 10379, 10526}","[6529, 10354, 666]","{6529, 10354}",0.666667,"{6529, 10354}",0.666667,0.888889
4,243,"{4544, 4992, 4834, 8900, 294, 3273, 5162, 1743...","[4544, 4992, 666]","{4544, 4992}",0.666667,"{4544, 4992}",0.666667,0.888889
...,...,...,...,...,...,...,...,...
10955,278668,{10605},"[10605, 666]",{10605},0.500000,{10605},0.500000,1.000000
10956,278694,"{8568, 8634, 5685, 6111}","[8568, 8634, 666]","{8568, 8634}",0.666667,"{8568, 8634}",0.666667,0.888889
10957,278843,"{4352, 3747, 8804, 7880, 8937, 6507, 111, 240,...","[4352, 3747, 666]","{4352, 3747}",0.666667,"{4352, 3747}",0.666667,0.888889
10958,278851,"{9663, 9907, 4214, 4607}","[9663, 9907, 666]","{9907, 9663}",0.666667,"{9907, 9663}",0.666667,0.888889


In [119]:
map_4 = purchases['ap@4'].mean()
map_4

0.9089010543388824

.